Election de 2022 premier tour

In [2]:
import numpy as np
import pandas as pd
import os
os.listdir()

['immigration.ipynb',
 'migrations',
 'election2017.ipynb',
 '.DS_Store',
 'alembic.ini',
 'economie',
 'enums',
 'depense.ipynb',
 'utils',
 'models',
 'election2012.ipynb',
 'exports',
 'labo1.ipynb',
 '.gitignore',
 'population.ipynb',
 'scripts',
 'db',
 'securite.ipynb',
 'chomages',
 'labo2.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'data',
 'chomage.ipynb']

#### Importer la database

Import des datasets.

In [3]:
df = pd.read_excel('./data/economie/menage/depenses.xlsx')

Visualise toutes les colones

In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Année,Dépenses pré-engagées,Dépenses liées au logement,Services de télévision et de télécommunications
0,1960,12.6,9.5,0.5
1,1961,13.1,10.0,0.5
2,1962,13.2,10.2,0.5
3,1963,13.9,10.7,0.5
4,1964,14.2,10.9,0.5
...,...,...,...,...
58,2018,29.1,22.7,2.1
59,2019,28.8,22.6,2.1
60,2020,28.5,22.5,2.1
61,2021,28.5,22.5,2.1


Renommer les colonnes 

In [9]:
df.columns = [
    "annee", "pre_engagees", "logement", "service_multimedia",
]
df.head()

,annee,pre_engagees,logement,service_multimedia
0,1960,12.6,9.5,0.5
1,1961,13.1,10.0,0.5
2,1962,13.2,10.2,0.5
3,1963,13.9,10.7,0.5
4,1964,14.2,10.9,0.5


Normalisation structurelle

In [29]:
# Conversion de l'année en int

df["annee"] = (
    df["annee"]
    .astype(str)
    .str.extract(r"(\d{4})")[0]  # sécurité si texte parasite
    .astype(int)
)


# Conversion des pourcentages en float

colonnes_pourcentage = [
    "pre_engagees", "logement", "service_multimedia",
]

for col in colonnes_pourcentage:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace("%", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
    )


In [22]:
df = df[(df["annee"] >= 2007) & (df["annee"] <= 2022)]

In [23]:
df.head()

,annee,pre_engagees,logement,service_multimedia
47,2007,28.4,20.9,2.8
48,2008,29.0,21.2,2.9
49,2009,28.9,21.5,2.9
50,2010,28.7,21.7,2.8
51,2011,28.6,21.8,2.7


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 47 to 62
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   annee               16 non-null     int64  
 1   pre_engagees        16 non-null     float64
 2   logement            16 non-null     float64
 3   service_multimedia  16 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 640.0 bytes


In [33]:
from db import SessionLocal
from models import MenageDepense

# Création d'une session
session = SessionLocal()

# Insertion des données du DataFrame
for row in df.itertuples(index=False):
    record = MenageDepense(
        annee=row.annee,
        pre_engagees=row.pre_engagees,
        logement=row.logement,
        service_multimedia=row.service_multimedia
    )
    session.add(record)

# Validation / commit
session.commit()

print(f"Insertion terminée pour les années {df['annee'].min()} à {df['annee'].max()}.")


ImportError: cannot import name 'MenageDepense' from 'models' (/workspace/models/__init__.py)